In [1]:
# Starters
import pandas as pd
from pathlib import Path
import os

In [2]:
# Spending File Pathing
health_spending_csv = Path("Resources/Health Care Expenditures per Capita by State of Residence raw data.csv")

#pd.read
spending_data = pd.read_csv(health_spending_csv,skiprows=[0,1])
spending_data = spending_data.dropna(axis=0,how='any',subset=["Health Spending per Capita"])
spending_data["Health Spending per Capita"] = spending_data["Health Spending per Capita"].str.replace(',', '').str.replace('$', '').astype(int)
spending_data["Health Spending per Capita"] = spending_data["Health Spending per Capita"].astype(float)
spending_data.head()

,Location,Health Spending per Capita
0,United States,9671.0
1,Alabama,8741.0
2,Alaska,13226.0
3,Arizona,8145.0
4,Arkansas,8853.0


In [3]:
# Coverage File Pathing
health_coverage_csv = Path("Resources/Health Insurance Coverage of the Total Population raw data #.csv")

#pd.read
coverage_data = pd.read_csv(health_coverage_csv,skiprows=[0,1])
coverage_data = coverage_data.dropna(axis=0,how='any',subset=["Employer"])
#coverage_data["Employer"] = coverage_data["Employer"].map("{:,}".format)
#coverage_data["Non-Group"] = coverage_data["Non-Group"].map("{:,}".format)
#coverage_data["Medicaid"] = coverage_data["Medicaid"].map("{:,}".format)
#coverage_data["Medicare"] = coverage_data["Medicare"].map("{:,}".format)
#coverage_data["Military"] = coverage_data["Military"].map("{:,}".format)
#coverage_data["Uninsured"] = coverage_data["Uninsured"].map("{:,}".format)
#coverage_data["Total"] = coverage_data["Total"].map("{:,}".format)
coverage_data.head()

,Location,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes
0,United States,158000000.0,18728800.0,63145700.0,45286700.0,4393600.0,29349300.0,319000000.0,1.0
1,Alabama,2250900.0,263400.0,929500.0,763800.0,99000.0,460400.0,4767100.0,NaN
2,Alaska,339800.0,24700.0,149400.0,70200.0,37100.0,80500.0,701700.0,NaN
3,Arizona,3202000.0,366500.0,1489600.0,1145300.0,105600.0,789100.0,7098000.0,NaN
4,Arkansas,1226300.0,157300.0,767000.0,464200.0,41800.0,265800.0,2922500.0,NaN


In [4]:
# Merging
merged_per_state = pd.merge( spending_data, coverage_data, how="left", on=["Location", "Location"] )
merged_per_state.head()

,Location,Health Spending per Capita,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes
0,United States,9671.0,158000000.0,18728800.0,63145700.0,45286700.0,4393600.0,29349300.0,319000000.0,1.0
1,Alabama,8741.0,2250900.0,263400.0,929500.0,763800.0,99000.0,460400.0,4767100.0,NaN
2,Alaska,13226.0,339800.0,24700.0,149400.0,70200.0,37100.0,80500.0,701700.0,NaN
3,Arizona,8145.0,3202000.0,366500.0,1489600.0,1145300.0,105600.0,789100.0,7098000.0,NaN
4,Arkansas,8853.0,1226300.0,157300.0,767000.0,464200.0,41800.0,265800.0,2922500.0,NaN


In [5]:
#Mortality File Pathing
hd_mortality_csv = Path("Resources/Heart Disease Mortality by State data table.csv")

#pd.read
mortaility_data = pd.read_csv(hd_mortality_csv)
mortaility_data = mortaility_data.loc[(mortaility_data["YEAR"] == 2019)]
mortaility_data = mortaility_data.reset_index(drop=True)
XX_to_full = {"AL":"Alabama",
              "AK":"Alaska",
              "AZ":"Arizona",
              "AR":"Arkansas",
              "CA":"California",
              "CO":"Colorado",
              "CT":"Connecticut",
              "DE":"Delaware",
              "DC":"District of Columbia",
              "FL":"Florida",
              "GA":"Georgia",
              "HI":"Hawaii",
              "ID":"Idaho",
              "IL":"Illinois",
              "IN":"Indiana",
              "IA":"Iowa",
              "KS":"Kansas",
              "KY":"Kentucky",
              "LA":"Louisiana",
              "ME":"Maine",
              "MD":"Maryland",
              "MA":"Massachusetts",
              "MI":"Michigan",
              "MN":"Minnesota",
              "MS":"Mississippi",
              "MO":"Missouri",
              "MT":"Montana",
              "NE":"Nebraska",
              "NV":"Nevada",
              "NH":"New Hampshire",
              "NJ":"New Jersey",
              "NM":"New Mexico",
              "NY":"New York",
              "NC":"North Carolina",
              "ND":"North Dakota",
              "MP":"Northern Mariana Islands",
              "OH":"Ohio",
              "OK":"Oklahoma",
              "OR":"Oregon",
              "PA":"Pennsylvania",
              "PR":"Puerto Rico",
              "RI":"Rhode Island",
              "SC":"South Carolina",
              "SD":"South Dakota",
              "TN":"Tennessee",
              "TX":"Texas",
              "UT":"Utah",
              "VT":"Vermont",
              "VA":"Virginia",
              "VI":"Virgin Islands",
              "WA":"Washington",
              "WV":"West Virginia",
              "WI":"Wisconsin",
              "WY":"Wyoming"}
mortaility_data["STATE"]=mortaility_data["STATE"].replace(XX_to_full)
mortaility_data = mortaility_data.rename(columns={"STATE":"Location"})
mortaility_data = mortaility_data.rename(columns={"YEAR":"Year"})
mortaility_data = mortaility_data.rename(columns={"RATE":"Deaths per 100k"})
mortaility_data = mortaility_data.rename(columns={"DEATHS":"Mortaility"})
mortaility_data["Mortaility"] = mortaility_data["Mortaility"].astype(float)
mortaility_data.head()

,Year,Location,Deaths per 100k,Mortaility
0,2019,Alabama,219.6,13448.0
1,2019,Alaska,129.7,843.0
2,2019,Arizona,134.0,12587.0
3,2019,Arkansas,226.5,8669.0
4,2019,California,136.9,62394.0


In [6]:
#final merge
full_combined =  pd.merge( merged_per_state, mortaility_data, how="left", on=["Location", "Location"])
full_combined = full_combined.dropna(axis=0,how='any',subset=["Year"])
full_combined.head()

,Location,Health Spending per Capita,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes,Year,Deaths per 100k,Mortaility
1,Alabama,8741.0,2250900.0,263400.0,929500.0,763800.0,99000.0,460400.0,4767100.0,NaN,2019.0,219.6,13448.0
2,Alaska,13226.0,339800.0,24700.0,149400.0,70200.0,37100.0,80500.0,701700.0,NaN,2019.0,129.7,843.0
3,Arizona,8145.0,3202000.0,366500.0,1489600.0,1145300.0,105600.0,789100.0,7098000.0,NaN,2019.0,134.0,12587.0
4,Arkansas,8853.0,1226300.0,157300.0,767000.0,464200.0,41800.0,265800.0,2922500.0,NaN,2019.0,226.5,8669.0
5,California,9628.0,18538700.0,2569600.0,9790000.0,4388900.0,350200.0,3005400.0,38642700.0,NaN,2019.0,136.9,62394.0


In [7]:
# Write to csv
output_path = os.path.join("DataFrames/full_combined.csv")

full_combined.to_csv(path_or_buf=output_path, index=False)

In [8]:
#national mean grouping

national_avg_spending = full_combined["Health Spending per Capita"].mean()
national_avg_employer = full_combined["Employer"].mean()
national_avg_non_group = full_combined["Non-Group"].mean()
national_avg_medicaid = full_combined["Medicaid"].mean()
national_avg_military = full_combined["Military"].mean()
national_avg_uninsured = full_combined["Uninsured"].mean()
national_avg_total = full_combined["Total"].mean()
national_avg_deaths_per = full_combined["Deaths per 100k"].mean()

national_avg_data = pd.DataFrame({"Average Health Spending per Capita" : [national_avg_spending],
                                 "Average Employer Coverage" : [national_avg_employer],
                                 "Average Non-Group Coverage" : [national_avg_non_group],
                                 "Average Medicaid Coverage" : [national_avg_medicaid],
                                 "Average Military Coverage" : [national_avg_military],
                                 "Average Uninsured" : [national_avg_uninsured],
                                 "Average Total Coverage" : [national_avg_total],
                                 "Average Deaths per 100k" : [national_avg_deaths_per]})
                                  
national_avg_data

,Average Health Spending per Capita,Average Employer Coverage,Average Non-Group Coverage,Average Medicaid Coverage,Average Military Coverage,Average Uninsured,Average Total Coverage,Average Deaths per 100k
0,9860.64,3159540.0,373706.0,1259500.0,87692.0,586498.0,6371556.0,163.688


In [9]:
location_count = len(full_combined["Location"].unique())
location_count

50

In [10]:
#national sum grouping

national_sum_spending = full_combined["Health Spending per Capita"].sum()
national_sum_employer = full_combined["Employer"].sum()
national_sum_non_group = full_combined["Non-Group"].sum()
national_sum_medicaid = full_combined["Medicaid"].sum()
national_sum_military = full_combined["Military"].sum()
national_sum_uninsured = full_combined["Uninsured"].sum()
national_sum_total = full_combined["Total"].sum()
national_sum_deaths = full_combined["Mortaility"].sum()

national_sum_data = pd.DataFrame({"Total Health Spending per Capita" : [national_sum_spending],
                                 "Total Employer Coverage" : [national_sum_employer],
                                 "Total Non-Group Coverage" : [national_sum_non_group],
                                 "Total Medicaid Coverage" : [national_sum_medicaid],
                                 "Total Military Coverage" : [national_sum_military],
                                 "Total Uninsured" : [national_sum_uninsured],
                                 "Total Total Coverage" : [national_sum_total],
                                 "Total Mortaility" : [national_sum_deaths]})
national_sum_data

,Total Health Spending per Capita,Total Employer Coverage,Total Non-Group Coverage,Total Medicaid Coverage,Total Military Coverage,Total Uninsured,Total Total Coverage,Total Mortaility
0,493032.0,157977000.0,18685300.0,62975000.0,4384600.0,29324900.0,318577800.0,657854.0


In [11]:
#percentages of data
health_coverage_p_csv = Path("Resources/Health Insurance Coverage of the Total Population raw data %.csv")

#pd.read
coverage_data_p = pd.read_csv(health_coverage_p_csv,skiprows=[0,1])
coverage_data_p = coverage_data_p.dropna(axis=0,how='any',subset=["Employer"])
coverage_data_p = coverage_data_p.rename(columns={"Employer":"Employer %"})
coverage_data_p["Employer %"] = coverage_data_p["Employer %"] * 100
coverage_data_p = coverage_data_p.rename(columns={"Non-Group":"Non-Group %"})
coverage_data_p["Non-Group %"] = coverage_data_p["Non-Group %"] * 100
coverage_data_p = coverage_data_p.rename(columns={"Medicaid":"Medicaid %"})
coverage_data_p["Medicaid %"] = coverage_data_p["Medicaid %"] * 100
coverage_data_p = coverage_data_p.rename(columns={"Medicare":"Medicare %"})
coverage_data_p["Medicare %"] = coverage_data_p["Medicare %"] * 100
coverage_data_p = coverage_data_p.rename(columns={"Military":"Military %"})
coverage_data_p["Military %"] = coverage_data_p["Military %"] * 100
coverage_data_p = coverage_data_p.rename(columns={"Uninsured":"Uninsured %"})
coverage_data_p["Uninsured %"] = coverage_data_p["Uninsured %"] * 100
coverage_data_p = coverage_data_p.rename(columns={"Total":"Total %"})
coverage_data_p["Total %"] = coverage_data_p["Total %"] * 100
coverage_data_p.head()

,Location,Employer %,Non-Group %,Medicaid %,Medicare %,Military %,Uninsured %,Total %,Footnotes
0,United States,49.6,5.9,19.8,14.2,1.4,9.2,100.0,1.0
1,Alabama,47.2,5.5,19.5,16.0,2.1,9.7,100.0,NaN
2,Alaska,48.4,3.5,21.3,10.0,5.3,11.5,100.0,NaN
3,Arizona,45.1,5.2,21.0,16.1,1.5,11.1,100.0,NaN
4,Arkansas,42.0,5.4,26.2,15.9,1.4,9.1,100.0,NaN


In [12]:
# Write to csv
output_path = os.path.join("DataFrames/coverage_data_p.csv")

coverage_data_p.to_csv(path_or_buf=output_path, index=False)

In [13]:
# Scatter plot 1
x1 = full_data["Deaths"]

y1 = full_data["Rate (per 100,000)"]
plt.figure(figsize=(14,9))
plt.scatter(x1, y1, alpha='0.9', edgecolors='black')
plt.xticks(rotation='vertical')

(slope, intercept, rvalue, pvalue, stderr) = linregress(x1, y1)
regress_values = x1 * slope + intercept
#line_eq = f"y = {round(slope,2)}x + {round(intercept,2)}"
#"y = " + (round(slope,2)) + "x + " + (round(intercept,2))

plt.plot(x1,regress_values,"r-")
#plt.annotate(line_eq,(5,200),fontsize=15,color="red")

plt.show()

NameError: name 'full_data' is not defined

In [ ]:
# bar graph
x_axis = np.arange(len(full_data))
tick_locations = [value for value in x_axis]

plt.figure(figsize=(14,6))
plt.bar(x_axis, full_data["Rate (per 100,000)"], align='center')
plt.xticks(tick_locations, full_data["State"], rotation="vertical")

plt.xlim(-1, len(x_axis))

plt.show()

In [ ]:
# bar graph
x_axis = np.arange(len(full_data))
tick_locations = [value for value in x_axis]

plt.figure(figsize=(14,6))
plt.bar(x_axis, full_data["Health Spending per Capita"], align='center')
plt.xticks(tick_locations, full_data["State"], rotation="vertical")

plt.xlim(-1, len(x_axis))

plt.show()